In [1]:
using TensorDecomposition, Symbolics, LinearAlgebra, Combinatorics, InvertedIndices, Einsum, IterTools, HomotopyContinuation

In [62]:
function kronMat(A::Matrix, d)
    if d == 1
        B = A
    else
        n, r = size(A)
        B = zeros(eltype(A), (n^d, r))
        for i=1:r 
            B[:, i] = kron(ntuple(x->A[:, i], d)...)
        end;
    end;
    return B
end;

function changeBasis(T, M)
    Tsize = size(T)
    n1 = Tsize[1]
    d = length(Tsize)
    
    n2 = size(M)[1]
    
    That = reshape(T, n1^d)
    return reshape(foldl(kron, repeat([M], d))*That, repeat([n2], d)...)
end;

function e(j, n)
    e = zeros(n)
    e[j] = 1
    return e
end;

function essentialVar(T, tol=1e-10)
    Tsize = size(T)
    n = Tsize[1]
    d = length(Tsize)
    
    That = reshape(T, (n, n^(d-1)))
    U, s, _ = svd(That)
    inds = (abs.(s) .> tol)
    if n == count(i->(i== 1), inds)
        return T, I
    else
        U = U[:, inds]
        return changeBasis(T, U'), U
    end
    
end;

function projectivize(L, A, d)
    return permutedims((Complex.(L) .^ (1/d))) .* A;
end;

function deprojectivize(A, d)
    B = A[1, :]
    return B .^ d, A ./ permutedims(B)
end;

function preprocess(T, tol=1e-10)
    T, U = essentialVar(T)
    
    Tsize = size(T)
    n = Tsize[1]
    d = length(Tsize)
    
    M = randn(n, n)
    M ./= sqrt(n)
    T = changeBasis(T, M)
    return T, U, M
end;

function multMon(x, j)
    c = copy(reverse(x))
    for i=1:length(c)
        if c[i] == 0
            c[i] = j
            break
        end
    end;
    return sort(reverse(c))
end;

function obtainDecomp(Ms, T)
    Ahat = zeros(ComplexF64, n, r)
    Ahat[1, :] = ones(r)
    for j=1:length(Ms)
        wj, Vj = eigen(Ms[j])
        argsort = sortperm(Vj[1, :], by=x -> (real(x), imag(x)), rev=true);
        Ahat[j+1, :] = wj[argsort]
    end
    Lhat = kronMat(Ahat, d)\reshape(T, n^d)
    return Lhat, Ahat
end;

function postprocess(Ahat, U, LT)
    return U*inv(LT)*Ahat
end;

function processLinEqs(linEqs, vars)
    A = []
    b = []
    
    z = zeros(length(vars))
    
    for linEq in linEqs
        coeffDict = HomotopyContinuation.to_dict(linEq, hs)
        push!(A, HomotopyContinuation.to_number.(sum([item.first * item.second for item in coeffDict])))
        push!(b, HomotopyContinuation.to_number(coeffDict[z]))
    end
    return Complex.(permutedims(reduce(hcat, A))), -Complex.(b)
    
end;

In [63]:
n_ = 4
d_ = 4;

In [64]:
r_ = 7;

In [65]:
# T_ = zeros(repeat([d_], d_)...)
# for perm in permutations(1:d_)
#     T_[perm...] = 1
# end

T_, A_, L = randomRankedTensor(n_+1, d_, r_, real=false)
A_ = projectivize(L, A_, d_)

T, U, LT = preprocess(T_);

In [66]:
tol = 1e-9;

In [67]:
Tsize = size(T)
n = Tsize[1]
d = length(Tsize)

r = r_;

In [68]:
if r <= binomial(n-1+Int(floor((d-1)/2)), n-1)
    
    # Regime where Jennrich might work
    
    Tcat = catMat(T, Int(floor(d/2)))
    H0 = Tcat[1:r, 1:r];
    H0_inv = inv(H0)

    D = Dict()
    first_r = []
    for (i, ind) in enumerate(with_replacement_combinations(0:n-1, d))
        if i <= r
            push!(first_r, ind)
        end
        D[Tuple(x for x in ind)] = i
    end

    Hs = []
    for i=1:n-1
        hankInds = []
        multMap = map(x -> multMon(x, i), first_r)
        for ind in multMap
            push!(hankInds, D[Tuple(x for x in ind)])
        end
        push!(Hs, Tcat[1:r, hankInds])
    end

    Ms = []
    for H in Hs
        push!(Ms, H*H0_inv)
    end;
    
    eqMats = []
    for i=1:n-1
        for j=i+1:n-1
            push!(eqMats, Ms[i]*Ms[j]-Ms[j]*Ms[i])
        end
    end
    if all([isapprox(eqMat, zeros(r, r), atol=tol) for eqMat in eqMats])
        Ahat_ = projectivize(obtainDecomp(Ms, T)..., d)
        Lhat, Ahat = deprojectivize(postprocess(Ahat_, U, LT), d)
    else
        error("Multiplication matrices do not commute")
    end
    
elseif r <= binomial(n-1+Int(floor(d/2)), n-1)
    Thank = hankMat(T);
    H0 = Thank[1:r, 1:r];
    H0 = convert(Matrix{eltype(T)}, H0)
    H0_inv = inv(H0)

    D = Dict()
    first_r = []
    for (i, ind) in enumerate(with_replacement_combinations(0:n-1, d))
        if i <= r
            push!(first_r, ind)
        end
        D[Tuple(x for x in ind)] = i
    end

    Hs = []
    for i=1:n-1
        hankInds = []
        multMap = map(x -> multMon(x, i), first_r)
        for ind in multMap
            push!(hankInds, D[Tuple(x for x in ind)])
        end
        push!(Hs, Thank[1:r, hankInds])
    end
    
    hs = collect(Set(vcat(HomotopyContinuation.variables.(Hs)...)))
    
    eqMats = []
    for i=1:n-1
        for j=i+1:n-1
            push!(eqMats, Hs[i]*H0_inv*Hs[j]-Hs[j]*H0_inv*Hs[i])
        end
    end
    
    gam = binomial(n-1+Int(floor((d-1)/2)), n-1)
    linInds = 1:gam
    quadInds = gam+1:r;
    
    linEqs = []
    quadEqs = []
    for eqMat in eqMats
        append!(linEqs, HomotopyContinuation.expand.(eqMat[linInds, quadInds]))
        append!(quadEqs, HomotopyContinuation.expand.(eqMat[quadInds, quadInds]))
    end
    
    A, b = processLinEqs(linEqs, hs);
    
    if rank(A) == length(hs)
        sol = A\b
    else
        F = System(vcat(linEqs, quadEqs), hs)
        result = solve(F)
        sols = solutions(result, only_nonsingular=false)
        if length(sols) == 0
            error("There are no solutions h such that the multiplication matrices commute.  The input r is likely not the correct rank.")
        end
        sol = sols[1]
    end
    
    Ms = []
    for H in Hs
        push!(Ms, evaluate.(H, hs => sol)*H0_inv)
    end
    eqMatsEval = []
    for i=1:n-1
        for j=i+1:n-1
            push!(eqMatsEval, Ms[i]*Ms[j]-Ms[j]*Ms[i])
        end
    end
    
    if all([isapprox(eqMat, zeros(r, r), atol=tol) for eqMat in eqMatsEval])
        Ahat_ = projectivize(obtainDecomp(Ms, T)..., d)
        Lhat, Ahat = deprojectivize(postprocess(Ahat_, U, LT), d)
    else
        error("Multiplication matrices do not commute")
    end
    
end;

In [69]:
(evaluate(Hs[1], hs=>sol)*H0_inv*det(H0)*evaluate(Hs[2], hs=>sol))[end, end]

0.08904222147990416 + 0.058994124124467764im

In [70]:
Hs[1]

7×7 Matrix{Expression}:
    1.24473302900335 + 2.02987156944498*im  …   -1.80420558783477 + 0.761552152683593*im
  0.974747434216298 + 0.583574338176092*im      -0.854579251028303 + 1.39838310875049*im
  -2.92293802794623 - 0.656672759949981*im         3.60277798218839 - 1.6103745238224*im
    0.901247983703431 + 1.1460102224077*im      -1.52141642586575 + 0.277167006453012*im
   2.00300404590654 - 0.301155783350463*im     -2.36804868614644 - 0.0565648933516582*im
 0.565909949213915 + 0.0436538535508631*im  …                                        h₇₂
  -1.80420558783477 + 0.761552152683593*im                                           h₇₅

In [71]:
Hs[3]

7×7 Matrix{Expression}:
   2.23868936481794 + 3.89618934844966*im  …   -2.30531812693442 - 0.796801552517627*im
   0.901247983703431 + 1.1460102224077*im      -1.52141642586575 + 0.277167006453012*im
  -4.69300095083048 - 3.47510923319299*im        3.18472427964564 + 0.79054655792462*im
  2.16143213949787 + 0.614133086516951*im      -1.23477412382999 - 0.279331027075073*im
   5.2620566049588 - 0.875144091686886*im     -0.940718594833105 - 0.213405217062764*im
 0.447052989858948 + 0.417345387506501*im  …                                        h₇₆
 -2.30531812693442 - 0.796801552517627*im                                           h₈₂

In [75]:
Hs[1][:, 6]

7-element Vector{Expression}:
 0.565909949213915 + 0.0436538535508631*im
  0.185614820198838 - 0.126483757523393*im
  -0.854579251028303 + 1.39838310875049*im
 0.204631343703062 + 0.0695189245091315*im
   0.294610836661797 - 1.12781241042831*im
                                       h₇₁
                                       h₇₂

In [76]:
Hs[3][:, 2]

7-element Vector{Expression}:
    0.901247983703431 + 1.1460102224077*im
  0.447052989858948 + 0.417345387506501*im
  -2.30531812693442 - 0.796801552517627*im
  0.389238924999511 + 0.157309926763368*im
   1.54202280315356 - 0.702809490930823*im
 0.204631343703062 + 0.0695189245091315*im
  -1.52141642586575 + 0.277167006453012*im

In [85]:
HomotopyContinuation.expand(det(Thank[[1, 2, 3, 4, 5, 6, 7, 16], [1, 2, 3, 4, 5, 6, 7, 8]]))

0.000478492772515438 - 0.000906593583749782*im + (0.00668578198377645 + 0.00916631109538457*im)*h₇₁ + (0.00104638677026969 - 0.000406574650583025*im)*h₇₂ + (-0.00256003592016896 - 0.0106829869832836*im)*h₇₃

In [90]:
HomotopyContinuation.expand.(permutedims(Hs[1][:, 6])*inv(H0)*det(H0)*Hs[3][:, 2]-permutedims(Hs[3][:, 6])*inv(H0)*det(H0)*Hs[1][:, 2])[1, 1]

-0.000478492771741959 + 0.000906593583754271*im + (-0.00668578198949094 - 0.00916631109472615*im)*h₇₁ + (-0.00104638677100752 + 0.000406574649249661*im)*h₇₂ + (0.00256003591456086 + 0.0106829869736923*im)*h₇₃ + (1.38777878078145e-17 - 5.20417042793042e-18*im)*h₇₆